In [1]:
import time
start_time = time.time()
%matplotlib qt
mpl_time = time.time()
import pyoptflight as pof
pof_main_time = time.time()
from pyoptflight import initialize as optinit
pof_init_time = time.time()
from pyoptflight import plotting as optplot
pof_plotting_time = time.time()
import numpy as np
np_time = time.time()

print(f'mpl: {mpl_time - start_time}')
print(f'pof main: {pof_main_time - mpl_time}')
print(f'pof init: {pof_init_time - pof_main_time}')
print(f'pof plotting: {pof_plotting_time - pof_init_time}')
print(f'np: {np_time - pof_plotting_time}')

mpl: 0.6012775897979736
pof main: 1.510624647140503
pof init: 0.0
pof plotting: 0.0
np: 0.0


In [2]:
kerbin = pof.Body("Kerbin")
kerbin.omega_0 = 0
vehicle = pof.Stage.load_vehicle('mintoc_multi')
# vehicle = pof.Stage.load_vehicle('mintoc_single')

vehicle[0].m_f = 8.4
vehicle[1].m_0 = 8.3

config = pof.SolverConfig(landing=False, 
                          T_min      = 1,
                          T_max      = 700,
                          max_iter   = 10000, 
                          solver_tol = 1e-3, 
                          N          = 50, 
                          T_init     = 100,
                          integration_method = 'RK4')

x0 = pof.LatLngBound(lat=0, lng=0, alt=0, vel = 1e-6, ERA0=0)

xf = pof.KeplerianBound(i    = np.deg2rad(20),
                        Ω    = np.deg2rad(0),
                        ω    = 0,
                        ha   = 120,
                        hp   = 120,
                        body = kerbin)

msolver = pof.Solver(kerbin, vehicle, config, x0, xf)

In [3]:
msolver.update_constraints(constraint_names='max_tau', new_values=0.10, new_enables=True)
msolver.update_constraints(constraint_names='max_body_rate_y', new_values=np.deg2rad(5), new_enables=True)
msolver.update_constraints(constraint_names='max_body_rate_z', new_values=np.deg2rad(5), new_enables=True)
msolver.update_constraints(constraint_names='f_min', new_values=0.15, new_enables=True)
msolver.update_constraints(constraint_names='max_alpha', new_values=np.deg2rad(10), new_enables=True)
msolver.update_constraints(constraint_names='max_q', new_values=0.2, new_enables=True)
msolver.constraints

[ConstraintSet(
     max_q           = Constraint(value=0.2, enabled=True),
     max_alpha       = Constraint(value=0.17453292519943295, enabled=True),
     max_body_rate_y = Constraint(value=0.08726646259971647, enabled=True),
     max_body_rate_z = Constraint(value=0.08726646259971647, enabled=True),
     max_tau         = Constraint(value=0.1, enabled=True),
     f_min           = Constraint(value=0.15, enabled=True)
 ),
 ConstraintSet(
     max_q           = Constraint(value=0.2, enabled=True),
     max_alpha       = Constraint(value=0.17453292519943295, enabled=True),
     max_body_rate_y = Constraint(value=0.08726646259971647, enabled=True),
     max_body_rate_z = Constraint(value=0.08726646259971647, enabled=True),
     max_tau         = Constraint(value=0.1, enabled=True),
     f_min           = Constraint(value=0.15, enabled=True)
 ),
 ConstraintSet(
     max_q           = Constraint(value=0.2, enabled=True),
     max_alpha       = Constraint(value=0.17453292519943295, enabled

In [4]:
msolver.create_nlp()

In [5]:
msolver.initialize_from_func(pof.gravity_turn, opts={'skew':True})

The minimum cost is 0.006308418983111963 rad
This is Ipopt version 3.14.11, running with linear solver MUMPS 5.4.1.

Number of nonzeros in equality constraint Jacobian...:     4937
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:     2976

Total number of variables............................:      907
                     variables with only lower bounds:      454
                variables with lower and upper bounds:      453
                     variables with only upper bounds:        0
Total number of equality constraints.................:      758
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  9.9711111e-01 2.66e+00 1.2

In [ ]:
pof.plot_solutions(msolver, colorscale=None, markers=None, show_actual_orbit=True, show_target_orbit=True, indices=[-1])

In [7]:
fig = pof.plot_flight_data(msolver, include=None)